# Classical Molecular Interaction Potentials tutorial using BioExcel Building Blocks (biobb)

***
This tutorial aims to illustrate the process of computing **classical molecular interaction potentials** from **protein structures**, step by step, using the **BioExcel Building Blocks library (biobb)**. Examples shown are **Molecular Interaction Potentials (MIPs) grids, protein-protein/ligand interaction potentials, and protein titration**. The particular structures used are the **Lysozyme** protein (PDB code [1AKI](https://www.rcsb.org/structure/1aki)), and a MD simulation of the complex formed by the **SARS-CoV-2 Receptor Binding Domain and the human Angiotensin Converting Enzyme 2** (PDB code [6VW1](https://www.rcsb.org/structure/6vw1)). 

The code wrapped is the ***Classical Molecular Interaction Potentials (CMIP)*** code:

**Classical molecular interaction potentials: Improved setup procedure in molecular dynamics simulations of proteins.**
*Gelpí, J.L., Kalko, S.G., Barril, X., Cirera, J., de la Cruz, X., Luque, F.J. and Orozco, M. (2001)*
*Proteins, 45: 428-437. https://doi.org/10.1002/prot.1159*
***

## Settings

### Biobb modules used

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_cmip](https://github.com/bioexcel/biobb_cmip): Tools to compute classical molecular interaction potentials from protein structures.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils): Tools to modify or extract information from a PDB structure.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to perform chemoinformatics on molecular structures.
 - [biobb_amber](https://github.com/bioexcel/biobb_amber): Tools to setup and simulate atomistic MD simulations using AMBER MD package.
  
### Auxiliar libraries used

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [plotly](https://plotly.com/python/): Python Open Source Graphing Library. 


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_cmip.git
cd biobb_wf_cmip
conda env create -f conda_env/environment.yml
conda activate biobb_wf_cmip
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-notebook biobb_wf_cmip/notebooks/biobb_wf_cmip.ipynb
  ``` 

***
## Pipeline steps
 1. [Input Parameters](#input)
 2. [Fetching PDB structure](#fetch)
 3. [CMIP PDB preparation (from PDB databank)](#preparePDB)
 4. [Structural water molecules & ions](#titration)
 5. [Molecular Interaction Potentials](#mips)
    1. [Positive MIP (MIP+)](#mip_pos) 
    2. [Negative MIP (MIP-)](#mip_neg) 
    3. [Neutral MIP (MIPn)](#mip_neutral) 
 6. [Protein-Ligand Interaction Energy (from PDB)](#prot-lig) 
    1. [Fetching PDB structure](#fetch2) 
    2. [Removing water molecules](#delWater)
    3. [Creating ligand topology](#ligandTop)
    4. [Generating system topology](#systemTop)
    5. [Minimizing the energy of the system](#minSystem)
 7. [Protein-Protein Interaction Energy (from MD)](#interaction)
    1. [CMIP PDB preparation (from MD)](#preparePDB_MD)
    2. [RDB Interaction Potential Energies](#RBDinteraction)
    3. [hACE2 Interaction Potential Energies](#hACE2interaction)
 8. [Questions & Comments](#questions)
 
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***

<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein structure (e.g. 1AKI)
 - **complexCode**: PDB code of the protein-ligand complex (e.g. 4HJO [*EGFR + Erlotinib*]) 
 - **ligandCode**: PDB code of the ligand (e.g. AQ4 [*Erlotinib*])
 - **mol_charge**: Charge of the small molecule
 - **MDCode**: Code of the Molecular Dynamics trajectory (e.g. RBD-hACE2)
     - **inputPDB_MD**: MD reference structure (PDB format)
     - **inputTOP_MD**: MD topology (Amber Parmtop7 format)

In [1]:
# TO BE REMOVED!!!

%load_ext autoreload
%autoreload 2

In [2]:
import nglview
import ipywidgets
import plotly
from plotly import subplots
import plotly.graph_objs as go

pdbCode = "1aki"       # Structure of the orthorhombic form of Hen Egg-white Lysozyme
complexCode = "4hjo"   # Crystal structure of the inactive EGFR tyrosine kinase domain with erlotinib
ligandCode = "AQ4"     # Erlotinib epidermal growth factor receptor (EGFR) tyrosine kinase inhibitor
mol_charge = 0         # Molecular charge for Erlotinib

MDCode = "RBD-hACE2-ZN" 
inputPDB_MD = "Files/" + MDCode + ".pdb" # Structure of the SARS-CoV-2 RBD-hACE2 complex
inputTOP_MD = "Files/" + MDCode + ".top" # MD Topology of the SARS-CoV-2 RBD-hACE2 complex

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [3]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode+'.pdb'
prop = {
    'pdb_code': pdbCode,
    'api_id' : 'mmb'
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2022-11-08 08:43:05,829 [MainThread  ] [INFO ]  Downloading: 1aki from: http://mmb.irbbarcelona.org/api/pdb/1aki/coords/?
2022-11-08 08:43:06,414 [MainThread  ] [INFO ]  Writting pdb to: 1aki.pdb
2022-11-08 08:43:06,419 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'MODEL', 'ENDMDL']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**: 

In [4]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="preparePDB"></a>
***
## CMIP PDB Preparation (from PDB structure)
**CMIP** tool needs additional information (e.g. charges, elements) to be included in the **structure PDB file** to properly run. A specific **BioBB building block** (prepare_pdb) is used in the next cell to prepare the **input PDB file**, adding this extra information. **Charges and elements** are taken from an internal **CMIP library** based on the **AMBER force fields**. 
***
**Building Blocks** used:
 - [prepare_pdb](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_pdb) from **biobb_cmip.cmip.prepare_pdb**
***

In [5]:
from biobb_cmip.cmip.prepare_pdb import prepare_pdb

cmipPDB = pdbCode + ".cmip.pdb"

prepare_pdb(input_pdb_path=downloaded_pdb,
            output_cmip_pdb_path=cmipPDB
)

2022-11-08 08:43:08,236 [MainThread  ] [INFO ]  Not using any container
2022-11-08 08:43:08,237 [MainThread  ] [INFO ]  check_structure -v -i 1aki.pdb -o 1aki.cmip.pdb --output_format cmip --non_interactive command_list --list 'water --remove yes; backbone --add_caps none; fixside --fix All; add_hydrogen --add_mode auto --add_charges CMIP'

2022-11-08 08:43:08,893 [MainThread  ] [INFO ]  Exit code 0

2022-11-08 08:43:08,895 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 1aki.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  129
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1001


Step 1: wa

0

<a id="titration"></a>
***
## Structural water molecules & ions
One of the many steps involved in the **MD structure setup process** is the addition of **solvent and counterions** (when working with explicit solvent). **Solvent molecules** and **counterions** are usually integrated on the **structure surface** in two steps:
- **Structural waters/ions**: A **first shell** of **water molecules and ions** is commonly added in the **energetically most favorable positions** on the surface of the structure. It is a computationally expensive process and is usually reduced to just tens of **water molecules** and **ions** (depending on the structure size).
- **Solvent box/ionic concentration**: A box of **solvent molecules** is created surrounding the original structure, and an additional number of **ions** are added until reaching a desired **ionic concentration**.

Whereas the second step is integrated in all the **MD packages**, the first one is rarely available. **CMIP** and the **biobb_titration building block** is helping in this task.   
***
**Building Blocks** used:
 - [titration](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.titration) from **biobb_cmip.cmip.titration**
 - [cat_pdb](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.cat_pdb) from **biobb_structure_utils.utils.cat_pdb**
***

<a id="run_titration"></a>
### Computing structural water molecules & ions positions
Computing the positions of **20 structural water molecules**, **5 positive** and **5 negative ions** in the most **energetically favourable** regions of the **structure surface**.

In [6]:
from biobb_cmip.cmip.titration import titration

wat_ions_pdb = pdbCode + ".wat_ions.pdb"
wat_ions_log = pdbCode + ".wat_ions.log"

prop = { 
#    'neutral' : True, # Can be also used to neutralize the system
    'num_positive_ions' : 5,
    'num_negative_ions' : 5,    
    'num_wats' : 20,
    'remove_tmp' : False
}

titration(input_pdb_path=cmipPDB,
          output_pdb_path=wat_ions_pdb,
          output_log_path=wat_ions_log,
          properties=prop)

2022-11-08 08:43:11,528 [MainThread  ] [INFO ]  Not using any container
2022-11-08 08:43:11,528 [MainThread  ] [INFO ]  titration -i 35abf65c-1797-4d7c-8dda-62372856028f/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs 1aki.cmip.pdb -outpdb 1aki.wat_ions

2022-11-08 08:43:29,011 [MainThread  ] [INFO ]  Exit code 0

2022-11-08 08:43:29,012 [MainThread  ] [INFO ]   =================================================
 =                                               =
 =                C M I P  (2.7.0)               =
 =          J. Ll. Gelpi, A. Morreale,           =
 =            F. J. Luque, M.Orozco              =
 =      Dept. Biochemistry. Univ. Barcelona      =
 =                  1999-2021                    =

  Code for ASA calculation by Juan F. Recio       

#T Run started at    8:43:11 h on  8-11-2022
 SIZES
 -----
 MAXTIP:           100
 MAXATH:            80000
 MAXATP x MAXCONF: 100000

 INPUT FILES
 -----------
    Calc. settings: 35abf65c-1797-4

0

<a id="catPDB_tit"></a>
### Adding structural water molecules & ions
Adding the 20 + 10 computed **structural water molecules** and **ions** to the original **PDB file**.

In [7]:
from biobb_structure_utils.utils.cat_pdb import cat_pdb

titPDB = pdbCode + ".tit.pdb"

cat_pdb(input_structure1=cmipPDB,
       input_structure2=wat_ions_pdb,
       output_structure_path=titPDB)

2022-11-08 08:43:39,566 [MainThread  ] [INFO ]  Removed: []


0

<a id="visTIT"></a>
### Visualizing structural water molecules & ions
Visualizing the recently added **structural water molecules and ions**. 

In [8]:
view = nglview.show_structure_file(titPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
#view.add_representation(repr_type='surface', selection='protein', radius='0.2', color='grey', opacity='0.2')
#view.add_representation(repr_type='licorice', radius='.5', selection='water')
view.add_representation(repr_type='spacefill', selection='water')
view.add_representation(repr_type='spacefill', selection='.Na', color='element')
view.add_representation(repr_type='spacefill', selection='.Cl', color='element')

#view.add_representation(repr_type='cartoon',selection='not het',colorScheme = 'atomindex')
#view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="mips"></a>
***
## Molecular Interaction Potentials (MIPs)
**Molecular interaction potentials (MIP)** are field properties arising from the interaction of a **probe** (e.g., methyl, proton or water) with a molecule. These are calculated in the surface of the molecule, with a grid defined around the structure.

**MIPs** are one of the most important molecular properties in the relationship between **molecular and binding data** (e.g. *3D Quantitative Structure-Activity Relationships, 3D-QSAR*), and is extensively applied in **drug discovery** processes.  

In this example, three different **MIPs** are used, with a **Water Oxygen atom** as a probe:
 - **Positive** MIP - highlighting the protein regions with **higher affinity** to **negatively charged groups**.
 - **Negative** MIP - highlighting the protein regions with **higher affinity** to **positively charged groups**.
 - **Neutral** MIP - highlighting the protein regions with **lower affinity** to **electrocharged groups**.
***
**Building Blocks** used:
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

<a id="mip_pos"></a>
### Positive MIP

In [89]:
from biobb_cmip.cmip.cmip import cmip

mip_pos_log = pdbCode + ".mip_pos.log"
mip_pos_cube = pdbCode + ".mip_pos.cube"

prop = { 
    'execution_type' : 'mip_pos',
    'remove_tmp' : False,
    #'binary_path' : "/Users/hospital/COVID/CMIP/CMIP-master-68aefeae92993bbaa7234a8f5010cc42264624d7/src/cmip"
}

cmip(input_pdb_path=cmipPDB,
          #output_pdb_path='output.pdb',  # If added, python crashes with output_pdb_path not exists!!
          output_log_path=mip_pos_log,
          output_cube_path=mip_pos_cube,
          properties=prop)

2022-10-04 10:25:19,235 [MainThread  ] [INFO ]  Not using any container
2022-10-04 10:25:19,235 [MainThread  ] [INFO ]  cmip -i 3e014057-308d-49ab-a486-eee80256857e/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs 1aki.cmip.pdb -cube 1aki.mip_pos.cube -o 1aki.mip_pos.log

2022-10-04 10:25:23,737 [MainThread  ] [INFO ]  Exit code 0



0

<a id="visMIP1"></a>
### Visualizing 3D structure
Visualizing the **positive MIP grid**, with protein regions with **higher affinity** to **negatively charged groups** highlighted.

In [10]:
view = nglview.show_structure_file(mip_pos_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="red")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="mip_neg"></a>
### Negative MIP

In [11]:
from biobb_cmip.cmip.cmip import cmip

mip_neg_log = pdbCode + ".mip_neg.log"
mip_neg_cube = pdbCode + ".mip_neg.cube"

prop = { 
    'execution_type' : 'mip_neg',
    #'binary_path' : "/Users/hospital/COVID/CMIP/CMIP-master-68aefeae92993bbaa7234a8f5010cc42264624d7/src/cmip"
}

cmip(input_pdb_path=cmipPDB,
          #output_pdb_path='output.pdb',  # If added, python crashes with output_pdb_path not exists!!
          output_log_path=mip_neg_log,
          output_cube_path=mip_neg_cube,
          properties=prop)

2022-09-28 13:06:14,570 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:06:14,571 [MainThread  ] [INFO ]  cmip -i 289b72a4-8831-42dd-9a91-5d271f4a6a11/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs 1aki.cmip.pdb -cube 1aki.mip_neg.cube -o 1aki.mip_neg.log

2022-09-28 13:06:22,676 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:06:22,677 [MainThread  ] [INFO ]  Removed: ['289b72a4-8831-42dd-9a91-5d271f4a6a11']


0

<a id="visMIP2"></a>
### Visualizing 3D structure
Visualizing the **negative MIP grid**, with protein regions with **higher affinity** to **positively charged groups** highlighted.

In [12]:
view = nglview.show_structure_file(mip_neg_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-5, color="red")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="mip_neutral"></a>
### Neutral MIP

In [13]:
from biobb_cmip.cmip.cmip import cmip

mip_neutral_log = pdbCode + ".mip_neutral.log"
mip_neutral_cube = pdbCode + ".mip_neutral.cube"

prop = { 
    'execution_type' : 'mip_neu'
}

cmip(input_pdb_path=cmipPDB,
          #output_pdb_path='output.pdb',  # If added, python crashes with output_pdb_path not exists!!
          output_log_path=mip_neutral_log,
          output_cube_path=mip_neutral_cube,
          properties=prop)

2022-09-28 13:06:23,187 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:06:23,187 [MainThread  ] [INFO ]  cmip -i 45b45ae8-f662-468a-89be-4bf5e723eade/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs 1aki.cmip.pdb -cube 1aki.mip_neutral.cube -o 1aki.mip_neutral.log

2022-09-28 13:06:31,632 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:06:31,634 [MainThread  ] [INFO ]  Removed: ['45b45ae8-f662-468a-89be-4bf5e723eade']


0

<a id="visMIP3"></a>
### Visualizing 3D structure
Visualizing the **neutral MIP grid**, with protein regions with **lower affinity** to **electrocharged groups** highlighted.

In [14]:
view = nglview.show_structure_file(mip_neutral_cube)
view.add_component(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_surface(isolevelType="value", isolevel=-1, color="grey")
view.component_1.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="visMIP4"></a>
### Visualizing 3D structure
Visualizing all **MIP grids**, for comparison purposes.

In [15]:
#Show different structures generated (for comparison)
view1 = nglview.show_structure_file(cmipPDB)
view1.add_component(mip_pos_cube)
view1.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view1.component_1.add_surface(isolevelType="value", isolevel=-5, color="blue")
view1.component_0.center()
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(cmipPDB)
view2.add_component(mip_neg_cube)
view2.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view2.component_1.add_surface(isolevelType="value", isolevel=-5, color="red")
view2.component_0.center()
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(cmipPDB)
view3.add_component(mip_neutral_cube)
view3.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view3.component_1.add_surface(isolevelType="value", isolevel=-1, color="grey")
view3.component_0.center()
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="interaction"></a>
***
## Interaction Potential Energies 

Closely related to the previous study of **Molecular Interaction Potentials**, the **Interaction Potential Energies** calculation computes the contributions to the **total energy** of the system from the different **interactions between the subunits of the molecule** considered. These **interaction energies** usually depend on the relation between the **charge** and **positions** of the units studied (e.g. *electrostatic, van der Waals*) and the **solvation energy** (energy released when a compound is dissolved in a solvent).

**Interaction Potential Energies** give useful insights in the **macromolecular interaction** process, with the possibility to identify **key residues** involved in the interaction, and thus being another key component of the **drug discovery** process.  

To illustrate the calculation of the **interaction potentials** between two subunits of a **structure complex** (e.g. protein-protein, protein-ligand), two different examples are used:

- **Epidermal Growth Factor Receptor** kinase domain with **Erlotinib** inhibitor.
- **SARS-CoV-2 Receptor Binding Domain** with **human Angiotensin Converting Enzyme 2**. 

<a id="prot-lig"></a>
***
## Protein-Ligand Interaction Energies

This example illustrates the steps needed to compute the **protein-ligand interaction energies** from a **crystal structure** of the **complex** taken directly from the **PDB data bank**. The particular example used is the **Epidermal Growth Factor Receptor** kinase domain (PDB code [4HJO](https://www.rcsb.org/structure/4HJO)) with **Erlotinib** inhibitor (PDB code [AQ4](https://www.rcsb.org/ligand/AQ4)). 

To properly compute the **interaction energies**, the **protein-ligand** complex needs to be pre-processed, adding the (typically) missing **hydrogen atoms**, finding out the **atomic charges** and **elements**, and **relaxing** the structure to avoid **steric issues** due to **crystallographic packing**. All these steps are performed in the following cells using **BioBB building blocks**

***

<a id="fetch2"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein molecule** from the RCSB PDB database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

In [16]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = complexCode+'.pdb'
prop = {
    'pdb_code': complexCode,
    'api_id' : 'mmb',
    'filter': ['ATOM', 'HETATM'],
}

#Create and launch bb
pdb(output_pdb_path=downloaded_pdb,
    properties=prop)

2022-09-28 13:06:32,267 [MainThread  ] [INFO ]  Downloading: 4hjo from: http://mmb.irbbarcelona.org/api/pdb/4hjo/coords/?
2022-09-28 13:06:32,371 [MainThread  ] [INFO ]  Writting pdb to: 4hjo.pdb
2022-09-28 13:06:32,372 [MainThread  ] [INFO ]  Filtering lines NOT starting with one of these words: ['ATOM', 'HETATM']


0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the downloaded/given **PDB structure** using **NGL**: 

In [17]:
# Show protein
view = nglview.show_structure_file(downloaded_pdb)
view.add_representation(repr_type='ball+stick', selection='hetero')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="delWater"></a>
***
## Removing Water Molecules
Removing the **water molecules** from the **downloaded/given structure**.<br>

***
**Building Blocks** used:
 - [remove_pdb_water](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.remove_pdb_water) from **biobb_structure_utils.utils**
***

In [18]:
# Import module
from biobb_structure_utils.utils.remove_pdb_water import remove_pdb_water

# Removing Waters:

# Create properties dict and inputs/outputs
nohoh_pdb = complexCode+'.noHOH.pdb'

#Create and launch bb
remove_pdb_water(
    input_pdb_path=downloaded_pdb,
    output_pdb_path=nohoh_pdb
)

2022-09-28 13:06:32,439 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:06:32,439 [MainThread  ] [INFO ]  check_structure -i 4hjo.pdb -o 4hjo.noHOH.pdb --force_save water --remove yes

2022-09-28 13:06:32,904 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:06:32,905 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.9.11                   =
=            P. Andrio, A. Hospital, G. Bayarri, J.L. Gelpi 2018-22           =

Structure 4hjo.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  313
 Num. residues with ins. codes:  0
 Num. HETATM residues:  35
 Num. ligands or modified residues:  1
 Num. water mol.:  34
 Num. atoms:  2264
Small mol ligands found
AQ4 A1001

Running water. Options: --remove yes
34 Water molecules detected
34 Water molecules removed
Final Num. models: 1
Final 

0

<a id="vis3D"></a>
### Visualizing 3D structure
Visualizing the **structure** without **water molecules** using **NGL**: 

In [19]:
# Show protein
view = nglview.show_structure_file(nohoh_pdb)
view.add_representation(repr_type='ball+stick', selection='hetero')
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="ligandTop"></a>
***
## Creating Ligand Topology
Obtaining parameters for the **small molecule** to be used in the **potential energy** calculations.<br>

***
**Building Blocks** used:
 - [extract_heteroatoms](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_heteroatoms) from **biobb_structure_utils.utils**
 - [reduce_add_hydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_add_hydrogens) from **biobb_chemistry.ambertools**
 - [acpype_params_ac](https://biobb-chemistry.readthedocs.io/en/latest/acpype.html#module-acpype.acpype_params_ac) from **biobb_chemistry.acpype**
***

### Step 1: Extracting Ligand

In [20]:
# Create Ligand system topology, STEP 1
# Extracting Ligand 
# Import module
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms

# Create properties dict and inputs/outputs
ligandFile = ligandCode+'.pdb'

prop = {
     'heteroatoms' : [{"name": ligandCode}]
}

extract_heteroatoms(
    input_structure_path=nohoh_pdb,
    output_heteroatom_path=ligandFile,
    properties=prop
)

2022-09-28 13:06:33,046 [MainThread  ] [INFO ]  Removed: []


0

### Step 2: Add hydrogen atoms to the ligand

In [21]:
# Create Ligand system topology, STEP 2
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

# Create prop dict and inputs/outputs
output_reduce_h = ligandCode+'.reduce.H.pdb' 

# Create and launch bb
reduce_add_hydrogens(
    input_path=ligandFile,
    output_path=output_reduce_h
) 

2022-09-28 13:06:33,068 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:06:33,068 [MainThread  ] [INFO ]  reduce /Users/hospital/BioBB/Notebooks_dev/biobb_wf_cmip/biobb_wf_cmip/notebooks/AQ4.pdb > AQ4.reduce.H.pdb

2022-09-28 13:06:33,359 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:06:33,360 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "/Users/hospital/BioBB/Notebooks_dev/biobb_wf_cmip/biobb_wf_cmip/notebooks/AQ4.pdb"
Database of HETATM connections: "/opt/anaconda3/envs/biobb_CMIP_tutorial//dat/reduce_wwPDB_het_dict.txt                                                                                                                                                                                                                        "
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored i

0

### Step 3: Generating ligand topology 

In [22]:
# Create Ligand system topology, STEP 3
# Acpype_params_gmx: Generation of topologies for AMBER with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_ac import acpype_params_ac

# Create prop dict and inputs/outputs
output_acpype_inpcrd = ligandCode+'params.inpcrd'
output_acpype_frcmod = ligandCode+'params.frcmod'
output_acpype_lib = ligandCode+'params.lib'
output_acpype_prmtop = ligandCode+'params.prmtop'
output_acpype = ligandCode+'params'

prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_ac(input_path=output_reduce_h, 
                output_path_inpcrd=output_acpype_inpcrd,
                output_path_frcmod=output_acpype_frcmod,
                output_path_lib=output_acpype_lib,
                output_path_prmtop=output_acpype_prmtop,
                 properties=prop)

2022-09-28 13:06:33,387 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:06:33,388 [MainThread  ] [INFO ]  acpype -i /Users/hospital/BioBB/Notebooks_dev/biobb_wf_cmip/biobb_wf_cmip/notebooks/AQ4.reduce.H.pdb -b AQ4params.h9rxa4 -n 0

2022-09-28 13:07:37,107 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:07:37,108 [MainThread  ] [INFO ]  ==========================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2022.6.6 (c) 2022 AWSdS |
==> ... charge set to 0
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
==> * Tleap OK *
==> Removing temporary files...
==> Using OpenBabel v.3.1.0

==> Writing NEW PDB file

==> Writing CNS/XPLOR files

==> Writing GROMACS files

==> Disambiguating lower and uppercase atomtypes in GMX top file, even if identical.

==> Writing GMX dihedrals for GMX 4.5 and higher.

==> Writi

0

### Visualizing 3D structures
Visualizing the **ligand** structures:

**Original ligand** PDB structure (left)
**Ligand** PDB structure with **hydrogen atoms** (right)

In [66]:
#Show different structures generated (for comparison)
view1 = nglview.show_structure_file(ligandFile)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['400px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(output_reduce_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['400px','400px'])
view2.camera='orthographic'
view2
ipywidgets.HBox([view1, view2])

<a id="systemTop"></a>
***
## Generating system topology
Generating the **system topology** for the **protein-ligand complex** using **tleap** from the **AmberTools package**. The **topology** will then be used to extract the needed **parameters** (e.g. atom charges) for the **potential energy** calculation.<br>

***
**Building Blocks** used:
 - [leap_gen_top](https://biobb-amber.readthedocs.io/en/latest/leap.html#module-leap.leap_gen_top) from **biobb_amber.leap**
***

In [24]:
# Import module
from biobb_amber.leap.leap_gen_top import leap_gen_top

# Create prop dict and inputs/outputs
output_pdb_path = 'structure.leap.pdb'
output_top_path = 'structure.leap.prmtop'
output_crd_path = 'structure.leap.crd'

prop = {
    "forcefield" : ["protein.ff14SB","gaff2"]
}

# Create and launch bb
leap_gen_top(input_pdb_path=nohoh_pdb,
           input_lib_path=output_acpype_lib,
           input_frcmod_path=output_acpype_frcmod,
           output_pdb_path=output_pdb_path,
           output_top_path=output_top_path,
           output_crd_path=output_crd_path,
           properties=prop)

2022-09-28 13:07:37,219 [MainThread  ] [INFO ]  Creating 21d43751-653a-4201-97df-50062798b95a temporary folder
2022-09-28 13:07:37,219 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:07:37,220 [MainThread  ] [INFO ]  tleap  -f 21d43751-653a-4201-97df-50062798b95a/leap.in

2022-09-28 13:07:37,676 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:07:37,677 [MainThread  ] [INFO ]  -I: Adding /opt/anaconda3/envs/biobb_CMIP_tutorial/dat/leap/prep to search path.
-I: Adding /opt/anaconda3/envs/biobb_CMIP_tutorial/dat/leap/lib to search path.
-I: Adding /opt/anaconda3/envs/biobb_CMIP_tutorial/dat/leap/parm to search path.
-I: Adding /opt/anaconda3/envs/biobb_CMIP_tutorial/dat/leap/cmd to search path.
-f: Source 21d43751-653a-4201-97df-50062798b95a/leap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./21d43751-653a-4201-97df-50062798b95a/leap.in
----- Source: /opt/anaconda3/envs/biobb_CMIP_tutorial/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /opt/anaconda3/env

0

<a id="minSystem"></a>
***
## Minimizing the energy of the system
Running a short **energetic minimization** to relax the system and remove **possible clashes** introduced by **crystallographic** constraints.<br>
                                                                                           
***
**Building Blocks** used:
 - [sander_mdrun](https://biobb-amber.readthedocs.io/en/latest/sander.html#module-sander.sander_mdrun) from **biobb_amber.sander**
 - [amber_to_pdb](https://biobb-amber.readthedocs.io/en/latest/ambpdb.html#module-ambpdb.amber_to_pdb) from **biobb_amber.ambpdb**
***

In [25]:
from biobb_amber.sander.sander_mdrun import sander_mdrun

trj_amber = "structure.amber.crd"
rst_amber = "structure.amber.rst"
log_amber = "structure.amber.log"

prop = {
    'simulation_type' : 'minimization',
    'mdin' : {
        'ntb' : 0,        # Periodic Boundary. No periodicity is applied and PME (Particle Mesh Ewald) is off.
        'cut' : 12,       # Nonbonded cutoff, in Angstroms.
        'maxcyc' : 500,   # Maximum number of cycles of minimization.
        'ncyc' : 50       # Minimization will be switched from steepest descent to conjugate gradient after ncyc cycles.
    }
}

sander_mdrun(
     input_top_path=output_top_path,
     input_crd_path=output_crd_path,
     output_traj_path=trj_amber,
     output_rst_path=rst_amber,
     output_log_path=log_amber,
     properties=prop
)

2022-09-28 13:07:37,704 [MainThread  ] [INFO ]  Creating f0b1ed44-e1a6-4d66-a4e1-f7db45e6b522 temporary folder
2022-09-28 13:07:37,705 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:07:37,705 [MainThread  ] [INFO ]  sander -O -i f0b1ed44-e1a6-4d66-a4e1-f7db45e6b522/sander.mdin -p structure.leap.prmtop -c structure.leap.crd -r structure.amber.rst -o structure.amber.log -x structure.amber.crd

2022-09-28 13:07:46,205 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:07:46,206 [MainThread  ] [INFO ]  Removed: ['f0b1ed44-e1a6-4d66-a4e1-f7db45e6b522', 'mdinfo']


0

### Extracting a PDB file from the minimized system 

In [26]:
from biobb_amber.ambpdb.amber_to_pdb import amber_to_pdb

pdb_amber_min = "structure.amber-min.pdb"

amber_to_pdb(
      input_top_path=output_top_path,
      input_crd_path=rst_amber,
      output_pdb_path=pdb_amber_min
)

2022-09-28 13:07:46,232 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:07:46,232 [MainThread  ] [INFO ]  ambpdb  -p structure.leap.prmtop -c structure.amber.rst >  structure.amber-min.pdb

2022-09-28 13:07:46,365 [MainThread  ] [INFO ]  Exit code 0



0

<a id="prepProtLigCMIP"></a>
***
## Preparing the structure for CMIP
Preparing the structure for the **CMIP calculations**, using the energetically relaxed **PDB structure** information.
***
**Building Blocks** used:
 - [prepare_structure](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_structure) from **biobb_cmip.cmip.prepare_structure**
***

In [27]:
from biobb_cmip.cmip.prepare_structure import prepare_structure

cmipPDB = complexCode + ".cmip.pdb"

prepare_structure(input_pdb_path=pdb_amber_min,
            input_topology_path=output_top_path,            
            output_cmip_pdb_path=cmipPDB
)

2022-09-28 13:07:46,390 [MainThread  ] [INFO ]  Reading: structure.leap.prmtop to extract charges
2022-09-28 13:07:46,629 [MainThread  ] [INFO ]  Reading: structure.leap.prmtop to extract elements
2022-09-28 13:07:47,041 [MainThread  ] [INFO ]  Removed: []


0

In [28]:
from biobb_structure_utils.utils.extract_residues import extract_residues

EGFR_Lig = complexCode + ".EGFR.lig.pdb"

prop = {
    'residues': [
        {
            'name': ligandCode
        }
    ]
}

extract_residues(input_structure_path=cmipPDB,
                 output_residues_path=EGFR_Lig,
                 properties=prop)

2022-09-28 13:07:47,122 [MainThread  ] [INFO ]  Removed: []


0

In [29]:
# Import module
from biobb_structure_utils.utils.remove_ligand import remove_ligand

# Create properties dict and inputs/outputs
cmipPDB_EGFR_Prot = complexCode+'.noLIG.pdb'

prop = {
    'ligand' : ligandCode
}

#Create and launch bb
remove_ligand(input_structure_path=cmipPDB,
    output_structure_path=cmipPDB_EGFR_Prot,
    properties=prop)

2022-09-28 13:07:47,142 [MainThread  ] [INFO ]  PDB format detected, removing all atoms from residues named AQ4
2022-09-28 13:07:47,145 [MainThread  ] [INFO ]  Removed: []


0

In [30]:
from biobb_cmip.cmip.ignore_residues import ignore_residues

cmipPDB_EGFR_Prot_ignored = complexCode + ".Prot_ignored.cmip.pdb"

protein_residues = list(range(1,279))
    
prop = {
    'residue_list': protein_residues # WARNING: Change residue number accordingly
}

ignore_residues(input_cmip_pdb_path = cmipPDB,
               output_cmip_pdb_path = cmipPDB_EGFR_Prot_ignored,
               properties = prop)

2022-09-28 13:07:47,166 [MainThread  ] [INFO ]  Residue list: [':1', ':2', ':3', ':4', ':5', ':6', ':7', ':8', ':9', ':10', ':11', ':12', ':13', ':14', ':15', ':16', ':17', ':18', ':19', ':20', ':21', ':22', ':23', ':24', ':25', ':26', ':27', ':28', ':29', ':30', ':31', ':32', ':33', ':34', ':35', ':36', ':37', ':38', ':39', ':40', ':41', ':42', ':43', ':44', ':45', ':46', ':47', ':48', ':49', ':50', ':51', ':52', ':53', ':54', ':55', ':56', ':57', ':58', ':59', ':60', ':61', ':62', ':63', ':64', ':65', ':66', ':67', ':68', ':69', ':70', ':71', ':72', ':73', ':74', ':75', ':76', ':77', ':78', ':79', ':80', ':81', ':82', ':83', ':84', ':85', ':86', ':87', ':88', ':89', ':90', ':91', ':92', ':93', ':94', ':95', ':96', ':97', ':98', ':99', ':100', ':101', ':102', ':103', ':104', ':105', ':106', ':107', ':108', ':109', ':110', ':111', ':112', ':113', ':114', ':115', ':116', ':117', ':118', ':119', ':120', ':121', ':122', ':123', ':124', ':125', ':126', ':127', ':128', ':129', ':130', ':131

In [31]:
from biobb_cmip.cmip.cmip import cmip

cmip_EGFR_box_log = complexCode + ".EGFR.box.log"
cmip_EGFR_box_out = complexCode + ".EGFR.box.byat.out"
cmip_EGFR_box_json = complexCode + ".EGFR.box.json"

prop = { 
    'execution_type' : 'check_only',
    'binary_path' : "/Users/hospital/BioBB/Programs/CMIP/dist/src/cmip",
    'remove_tmp':False,
    #'box_size_factor': 0.97,
    'params' : {
         'perfill' : 0.8
    }
}

cmip(input_pdb_path=cmipPDB,
     output_log_path=cmip_EGFR_box_log,
     output_byat_path=cmip_EGFR_box_out,
     output_json_box_path=cmip_EGFR_box_json,
     properties=prop)

2022-09-28 13:07:47,391 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:07:47,391 [MainThread  ] [INFO ]  /Users/hospital/BioBB/Programs/CMIP/dist/src/cmip -i 781007cc-1d1d-4431-b8da-727c58ff90b9/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs 4hjo.cmip.pdb -byat 4hjo.EGFR.box.byat.out -o 4hjo.EGFR.box.log -l cd497557-5c07-48fa-a91d-e44e8106f8ea/key_value_cmip_log.log

2022-09-28 13:07:47,455 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:07:47,456 [MainThread  ] [INFO ]  STOP 0



0

In [32]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(cmip_EGFR_box_log, cmipPDB)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# vertices box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [35]:
from biobb_cmip.cmip.cmip import cmip

EGFR_energies_log = complexCode + ".EGFR.energies.log"
EGFR_byat_out = complexCode + ".EGFR.energies.byat.out"

prop = { 
    'execution_type' : 'pb_interaction_energy'
}

cmip(input_pdb_path=cmipPDB_EGFR_Prot_ignored,
     input_probe_pdb_path=cmipPDB_EGFR_Prot,
          output_log_path=EGFR_energies_log,
          output_byat_path=EGFR_byat_out,
          properties=prop)

2022-09-28 13:11:33,652 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:11:33,652 [MainThread  ] [INFO ]  cmip -i e8213e6d-62f5-428b-bd3a-e1bd1107d943/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs 4hjo.Prot_ignored.cmip.pdb -pr 4hjo.noLIG.pdb -byat 4hjo.EGFR.energies.byat.out -o 4hjo.EGFR.energies.log

2022-09-28 13:11:37,306 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:11:37,308 [MainThread  ] [INFO ]  Removed: ['e8213e6d-62f5-428b-bd3a-e1bd1107d943']


0

In [36]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat

atom_list, energy_dict = get_energies_byat(EGFR_byat_out, cutoff=50)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES&VDW'])],
       "layout": go.Layout(title="CMIP Interaction Potential", 
                           xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)

In [68]:
import plotly
import plotly.graph_objs as go
import re
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(EGFR_byat_out, cutoff=55)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES&VDW'])],
       "layout": go.Layout(title="CMIP Interaction Potential", 
                           xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)

energy_cutoff = -1.5
identified_residues = [res_list[energy_dict['ES&VDW'].index(item)] 
                       for item in energy_dict['ES&VDW'] if item < energy_cutoff]
identified_residues_ngl = str([int(re.sub(r'[A-Z]+ ', '', item)) for item in identified_residues]).replace(',','')
print (identified_residues_ngl)

[16 37 84 86 87 89 90 138 148 149]


In [69]:
# Show protein
view = nglview.show_structure_file(cmipPDB)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='all',color='sstruc',opacity=0.3)
view.add_representation(repr_type='ball+stick', selection=ligandCode)
view.add_representation(repr_type='licorice', selection=identified_residues_ngl)
view.center(ligandCode)
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="prot-prot"></a>
***
## Protein-Protein Interaction Energies
XXX
***
**Building Blocks** used:
 - [extract_chain](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_chain) from **biobb_structure_utils.utils.extract_chain**
 - [prepare_structure](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_structure) from **biobb_cmip.cmip.prepare_structure**
 - [cmip](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.cmip) from **biobb_cmip.cmip.cmip**
***

<a id="preparePDB_MD"></a>
***
## CMIP PDB Preparation (from Molecular Dynamics topology)
When working with **structure conformations** taken from **MD simulations**, it is recommended to use the **charges, atom types and elements** considered in the simulation. Usually this information is stored in the so-called ***topology*** files. A specific **building block** (***prepare_structure***) is available to extract these information from an **MD topology file** and use it for the **CMIP calculations**.
 
The next cells are taking one frame of the **MD simulation**, splitting the subunits (chains) in two different **PDB files**, and preparing them to be used in **CMIP**, taking the **charges and elements** used in the simulations from the **MD topology file**. 
***
**Building Blocks** used:
 - [extract_chain](https://biobb-structure-utils.readthedocs.io/en/latest/utils.html#module-utils.extract_chain) from **biobb_structure_utils.utils.extract_chain**
 - [prepare_structure](https://biobb-cmip.readthedocs.io/en/latest/cmip.html#module-cmip.prepare_structure) from **biobb_cmip.cmip.prepare_structure**
***

<a id="visRBD-hACE2"></a>
### Visualizing 3D structure
Visualizing the original **SARS-CoV-2 Receptor Binding Domain** (blue chain) and the **human Angiotensin Converting Enzyme 2** (red chain) structure complex, taken from a **MD simulation trajectory**.

In [39]:
view = nglview.show_structure_file(inputPDB_MD)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='chainname')
view._remote_call('setSize', target='Widget', args=['','400px'])
view

NGLWidget()

### Preparing structure
Preparing the structure for the **CMIP calculations**, using the original **MD topology file** information.

In [40]:
from biobb_cmip.cmip.prepare_structure import prepare_structure

cmipPDB_MD = MDCode + ".cmip.pdb"

prepare_structure(input_pdb_path=inputPDB_MD,
                  input_topology_path=inputTOP_MD,
            output_cmip_pdb_path=cmipPDB_MD
)

2022-09-28 13:11:47,717 [MainThread  ] [INFO ]  Reading: Files/RBD-hACE2-ZN.top to extract charges
2022-09-28 13:12:09,507 [MainThread  ] [INFO ]  Reading: Files/RBD-hACE2-ZN.top to extract elements
2022-09-28 13:12:39,886 [MainThread  ] [INFO ]  Removed: []


0

### Extracting chains 
Saving **hACE2** (chain A) and **RBD** (chain B) in two different **PDB files**. 

In [41]:
from biobb_structure_utils.utils.extract_chain import extract_chain

cmipPDB_MD_hACE2 = MDCode + ".hACE2.cmip.pdb"
cmipPDB_MD_RBD = MDCode + ".RBD.cmip.pdb"

prop = {
    'chains': [ 'A' ],
    'permissive' : True    
}
extract_chain(input_structure_path=cmipPDB_MD,
            output_structure_path=cmipPDB_MD_hACE2,
            properties=prop)

prop = {
    'chains': [ 'B' ],
    'permissive' : True
}
extract_chain(input_structure_path=cmipPDB_MD,
            output_structure_path=cmipPDB_MD_RBD,
            properties=prop)

2022-09-28 13:12:39,916 [MainThread  ] [INFO ]  Selected Chains: A
2022-09-28 13:12:39,916 [MainThread  ] [INFO ]  Warning: Use permissive=True is a risky option use it under your own responsability
2022-09-28 13:12:39,926 [MainThread  ] [INFO ]  Removed: []
2022-09-28 13:12:39,929 [MainThread  ] [INFO ]  Selected Chains: B
2022-09-28 13:12:39,929 [MainThread  ] [INFO ]  Warning: Use permissive=True is a risky option use it under your own responsability
2022-09-28 13:12:39,938 [MainThread  ] [INFO ]  Removed: []


In [70]:
#Show different structures generated 
view1 = nglview.show_structure_file(cmipPDB_MD_hACE2)
view1.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view1.component_0.center()
view1._remote_call('setSize', target='Widget', args=['400px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(cmipPDB_MD_RBD)
view2.component_0.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view2.component_0.center()
view2._remote_call('setSize', target='Widget', args=['400px','400px'])
view2.camera='orthographic'
view2
ipywidgets.HBox([view1, view2])

In [43]:
from biobb_cmip.cmip.cmip import cmip

cmip_RBD_box_log = "RBD.box.log"
cmip_RBD_box_out = "RBD.box.byat.out"
cmip_RBD_box_json = "RBD.box.json"

prop = { 
    'execution_type' : 'check_only',
    'binary_path' : "/Users/hospital/BioBB/Programs/CMIP/dist/src/cmip",
    'remove_tmp':False,
    #'box_size_factor': 0.97,
    'params' : {
         'perfill' : 0.8
    }
}

cmip(input_pdb_path=cmipPDB_MD_RBD,
     output_log_path=cmip_RBD_box_log,
     output_byat_path=cmip_RBD_box_out,
     output_json_box_path=cmip_RBD_box_json,
     properties=prop)

2022-09-28 13:12:40,062 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:12:40,062 [MainThread  ] [INFO ]  /Users/hospital/BioBB/Programs/CMIP/dist/src/cmip -i 6f3ae41f-b67f-4da9-946c-bb19e8857aa6/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs RBD-hACE2-ZN.RBD.cmip.pdb -byat RBD.box.byat.out -o RBD.box.log -l f70e8295-4d38-4769-97aa-d9d6d9f5a0cc/key_value_cmip_log.log

2022-09-28 13:12:40,101 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:12:40,102 [MainThread  ] [INFO ]  STOP 0



0

In [71]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(cmip_RBD_box_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# vertices box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [45]:
from biobb_cmip.cmip.cmip import cmip

cmip_hACE2_box_log = "hACE2.box.log"
cmip_hACE2_box_out = "hACE2.box.byat.out"
cmip_hACE2_box_json = "hACE2.box.json"

prop = { 
    'execution_type' : 'check_only',
    'binary_path' : "/Users/hospital/BioBB/Programs/CMIP/dist/src/cmip",
    'remove_tmp':False,
    #'box_size_factor': 0.97,
    'params' : {
         'perfill' : 0.8,
    }
}

cmip(input_pdb_path=cmipPDB_MD_hACE2,
     output_log_path=cmip_hACE2_box_log,
     output_byat_path=cmip_hACE2_box_out,
     output_json_box_path=cmip_hACE2_box_json,
     properties=prop)

2022-09-28 13:12:40,247 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:12:40,247 [MainThread  ] [INFO ]  /Users/hospital/BioBB/Programs/CMIP/dist/src/cmip -i e4bb780d-c3bb-4d80-abc3-5d8d7cd29531/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs RBD-hACE2-ZN.hACE2.cmip.pdb -byat hACE2.box.byat.out -o hACE2.box.log -l 8b45a5d5-c253-4224-98c9-6891020bc2b9/key_value_cmip_log.log

2022-09-28 13:12:40,307 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:12:40,308 [MainThread  ] [INFO ]  STOP 0



0

In [72]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(cmip_hACE2_box_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# vertices box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

In [47]:
from biobb_cmip.cmip.cmip import cmip

cmip_COMPLEX_box_log = "COMPLEX.box.log"
cmip_COMPLEX_box_out = "COMPLEX.box.byat.out"
cmip_COMPLEX_box_json = "COMPLEX.box.json"

prop = { 
    'execution_type' : 'check_only',
    'binary_path' : "/Users/hospital/BioBB/Programs/CMIP/dist/src/cmip",
    'remove_tmp':False,
    #'box_size_factor': 0.97,
    'params' : {
         'perfill' : 0.6,
    }
}

cmip(input_pdb_path=cmipPDB_MD,
     output_log_path=cmip_COMPLEX_box_log,
     output_byat_path=cmip_COMPLEX_box_out,
     output_json_box_path=cmip_COMPLEX_box_json,
     properties=prop)

2022-09-28 13:12:40,433 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:12:40,433 [MainThread  ] [INFO ]  /Users/hospital/BioBB/Programs/CMIP/dist/src/cmip -i fd230a52-c5f2-492c-902c-9fc196164657/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs RBD-hACE2-ZN.cmip.pdb -byat COMPLEX.box.byat.out -o COMPLEX.box.log -l 8442fafc-0533-49f8-9a40-f6e9abd06275/key_value_cmip_log.log

2022-09-28 13:12:40,610 [MainThread  ] [INFO ]  Exit code 0

2022-09-28 13:12:40,611 [MainThread  ] [INFO ]  STOP 0



0

In [73]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(cmip_COMPLEX_box_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# vertices box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="RBDinteraction"></a>
***
## RDB Interaction Potential Energies 
The first analysis computes the **interaction potential energies** for the **RBD atoms** (CMIP input probe) with respect to the **hACE2 enzyme** (CMIP input protein). 

In [49]:
from biobb_cmip.cmip.ignore_residues import ignore_residues

cmipPDB_MD_RBD_ignored = MDCode + ".RBD_ignored.cmip.pdb"

prop = {
    'residue_list': "B:"
}

ignore_residues(input_cmip_pdb_path = cmipPDB_MD,
               output_cmip_pdb_path = cmipPDB_MD_RBD_ignored,
               properties = prop)

2022-09-28 13:12:40,740 [MainThread  ] [INFO ]  Residue list: ['B:']
2022-09-28 13:12:40,762 [MainThread  ] [INFO ]  3000 residues have been marked
2022-09-28 13:12:40,763 [MainThread  ] [INFO ]  Removed: []


In [50]:
from biobb_cmip.cmip.cmip import cmip

RBD_energies_log = "RBD.energies.log"
RBD_byat_out = "RBD.energies.byat.out"
output_RBD_box_json = "RBD.energies.box.output.json"
output_COMPLEX_box_json = "COMPLEX.energies.box.output.json"

prop = { 
    'execution_type' : 'pb_interaction_energy',
    'remove_tmp' : False,
    'params' : {
        'pbfocus' : 1,  # ara està per defecte a energy, però això canviarà a pb_interaction_energy
    }
}

cmip(input_pdb_path=cmipPDB_MD_RBD_ignored,
     input_probe_pdb_path=cmipPDB_MD_RBD,
     input_json_box_path=cmip_RBD_box_json,
     input_json_external_box_path=cmip_COMPLEX_box_json,
     output_json_box_path=output_RBD_box_json,
     output_json_external_box_path=output_COMPLEX_box_json,
#          output_pdb_path='output.pdb', # If added, python crashes with output_pdb_path not exists!!
          output_log_path=RBD_energies_log,
          output_byat_path=RBD_byat_out,
          properties=prop)


2022-09-28 13:12:40,789 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:12:40,789 [MainThread  ] [INFO ]  cmip -i 2d6d3132-cbfa-400a-9a79-3ef28c2961da/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs RBD-hACE2-ZN.RBD_ignored.cmip.pdb -pr RBD-hACE2-ZN.RBD.cmip.pdb -byat RBD.energies.byat.out -o RBD.energies.log -l 53f7393b-b172-4537-a13d-46929f8cf4fe/key_value_cmip_log.log

2022-09-28 13:14:27,469 [MainThread  ] [INFO ]  Exit code 0



0

<a id="visBOX1"></a>
### Visualizing CMIP Box
Visualizing the **box** used by **CMIP** to compute the **Interaction Potential Energies** (taken from the log file). It is important to check that the box includes the whole **interaction region**, which is the region of interest. 

In [74]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(output_COMPLEX_box_json, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# vertices box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="plotRBD_atoms"></a>
### Interaction energy by atom
Visualizing the **interaction potential energies** computed by **CMIP**. The plot shows **interactions energies** (in kcal/mol, Y axis) for **each of the atoms** of the **RBD protein** (X axis). 

In [52]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat

atom_list, energy_dict = get_energies_byat(RBD_byat_out, cutoff=55)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES&VDW'])],
       "layout": go.Layout(title="CMIP Interaction Potential", 
                           xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)

<a id="plotRBD_residues"></a>
### Interaction energy by residue
Visualizing the **interaction potential energies** computed by **CMIP**. The plot shows **interactions energies** (in kcal/mol, Y axis) for **each of the residues** (computed summing the contributions of all atoms included in the residue) of the **RBD protein** (X axis). 

In [76]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(RBD_byat_out, cutoff=55)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES&VDW'])],
       "layout": go.Layout(title="CMIP Interaction Potential", 
                           xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)

energy_cutoff = -1.5
identified_residues = [res_list[energy_dict['ES&VDW'].index(item)] 
                       for item in energy_dict['ES&VDW'] if item < energy_cutoff]
identified_residues_ngl = str([int(re.sub(r'[A-Z]+', '', item)) for item in identified_residues]).replace(',','')

In [77]:
# Show protein
view = nglview.show_structure_file(cmipPDB_MD)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='all',color='sstruc',opacity=0.3)
view.add_representation(repr_type='licorice', selection=identified_residues_ngl)
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="hACE2interaction"></a>
***
## hACE2 Interaction Potential Energies 
The first analysis computes the **interaction potential energies** for the **hACE2 atoms** (CMIP input probe) with respect to the **RBD receptor** (CMIP input protein). 

In [55]:
from biobb_cmip.cmip.ignore_residues import ignore_residues

cmipPDB_MD_hACE2_ignored = MDCode + ".hACE2_ignored.cmip.pdb"

prop = {
    'residue_list': "A:"
}

ignore_residues(input_cmip_pdb_path = cmipPDB_MD,
               output_cmip_pdb_path = cmipPDB_MD_hACE2_ignored,
               properties = prop)

2022-09-28 13:14:27,823 [MainThread  ] [INFO ]  Residue list: ['A:']
2022-09-28 13:14:27,855 [MainThread  ] [INFO ]  9584 residues have been marked
2022-09-28 13:14:27,856 [MainThread  ] [INFO ]  Removed: []


In [56]:
from biobb_cmip.cmip.cmip import cmip

hACE2_energies_log = "hACE2.energies.log"
hACE2_byat_out = "hACE2.energies.byat.out"
output_hACE2_box_json = "hACE2.energies.box.output.json"
output_COMPLEX_2_box_json = "COMPLEX_2.energies.box.output.json"

prop = { 
    'execution_type' : 'pb_interaction_energy',
    'remove_tmp' : False,
}

cmip(input_pdb_path=cmipPDB_MD_hACE2_ignored,
     input_probe_pdb_path=cmipPDB_MD_hACE2,
     input_json_box_path=cmip_hACE2_box_json,
     input_json_box_external_path=cmip_COMPLEX_box_json,
     output_json_box_path=output_hACE2_box_json,
     output_json_external_box_path=output_COMPLEX_2_box_json,
#          output_pdb_path='output.pdb', # If added, python crashes with output_pdb_path not exists!!
          output_log_path=hACE2_energies_log,
          output_byat_path=hACE2_byat_out,
          properties=prop)

2022-09-28 13:14:27,887 [MainThread  ] [INFO ]  Not using any container
2022-09-28 13:14:27,888 [MainThread  ] [INFO ]  cmip -i 2936c057-22db-416d-86a1-c736138a71ca/params -vdw /opt/anaconda3/envs/biobb_CMIP_tutorial/share/cmip/dat/vdwprm -hs RBD-hACE2-ZN.hACE2_ignored.cmip.pdb -pr RBD-hACE2-ZN.hACE2.cmip.pdb -byat hACE2.energies.byat.out -o hACE2.energies.log -l 95daa1dc-31d6-41a3-8f63-f9269ceb43b7/key_value_cmip_log.log

2022-09-28 13:15:41,220 [MainThread  ] [INFO ]  Exit code 0



0

<a id="visBOX2"></a>
### Visualizing CMIP Box
Visualizing the **box** used by **CMIP** to compute the **Interaction Potential Energies** (taken from the log file). It is important to check that the box includes the whole **interaction region**, which is the region of interest. 

In [57]:
import nglview as nv
from biobb_cmip.utils.representation import create_box_representation

boxedFilename, atomPair = create_box_representation(hACE2_energies_log, inputPDB_MD)
# Represent the new file in ngl
view = nv.show_structure_file(boxedFilename, default=False)
# Structure
view.add_representation(repr_type='cartoon', selection='not het', color='#cccccc', opacity=.2)
# vertices box
view.add_representation(repr_type='ball+stick', selection='9999', aspectRatio = 10)
# lines box
view.add_representation(repr_type='distance', atomPair= atomPair, labelColor= 'transparent', color= 'black')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

<a id="plotRBD_atoms"></a>
### Interaction energy by atom
Visualizing the **interaction potential energies** computed by **CMIP**. The plot shows **interactions energies** (in kcal/mol, Y axis) for **each of the atoms** of the **hACE2 protein** (X axis). 

In [78]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byat

atom_list, energy_dict = get_energies_byat(hACE2_byat_out, cutoff=55)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=atom_list, y=energy_dict['ES&VDW'])],
       "layout": go.Layout(title="CMIP Interaction Potential", 
                           xaxis=dict(title = "Atom Number"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)

<a id="plotRBD_residues"></a>
### Interaction energy by residue
Visualizing the **interaction potential energies** computed by **CMIP**. The plot shows **interactions energies** (in kcal/mol, Y axis) for **each of the residues** (computed summing the contributions of all atoms included in the residue) of the **hACE2 protein** (X axis). 

In [79]:
import plotly
import plotly.graph_objs as go
from biobb_cmip.utils.representation import get_energies_byres


res_list, energy_dict = get_energies_byres(hACE2_byat_out, cutoff=55)

plotly.offline.init_notebook_mode(connected=True)

fig = {"data": [go.Scatter(x=res_list, y=energy_dict['ES&VDW'])],
       "layout": go.Layout(title="CMIP Interaction Potential", 
                           xaxis=dict(title = "Residue ID"), 
                           yaxis=dict(title = "Potential Energy Kcal/mol"))}

plotly.offline.iplot(fig)

energy_cutoff = -1.5
identified_residues = [res_list[energy_dict['ES&VDW'].index(item)] 
                       for item in energy_dict['ES&VDW'] if item < energy_cutoff]
identified_residues_ngl = str([int(re.sub(r'[A-Z]+', '', item)) for item in identified_residues]).replace(',','')

In [80]:
# Show protein
view = nglview.show_structure_file(cmipPDB_MD)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='all',color='sstruc',opacity=0.3)
view.add_representation(repr_type='ball+stick', selection=ligandCode)
view.add_representation(repr_type='licorice', selection=identified_residues_ngl)
#view.center(ligandCode)
view._remote_call('setSize', target='Widget', args=['','600px'])
view

NGLWidget()

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
